In [126]:
import parse

def parse_line(s):
    
    a,b = s.split(": ")

    # Blueprint number
    n = int(a.split(" ")[1])

    r = parse.parse("Each ore robot costs {:d} ore. Each clay robot costs {:d} ore. Each obsidian robot costs {:d} ore and {:d} clay. Each geode robot costs {:d} ore and {:d} obsidian.", b)

    return [n, *list(r)]

input = [parse_line(l) for l in open("inputs/19-input.txt").read().splitlines()]

In [132]:
from functools import lru_cache

@lru_cache(maxsize=14000000)
def max_geodes(
    t, 
    ore, clay, obs,
    o_rob, c_rob, obs_rob,
    o_cost, c_cost, obs_o_cost, obs_c_cost, g_o_cost, g_obs_cost
):
    
    resources = [ore, clay, obs]
    robots = [o_rob, c_rob, obs_rob]
    costs = [o_cost, c_cost, obs_o_cost, obs_c_cost, g_o_cost, g_obs_cost]

    # Most expensive robot (in ore)
    max_ore_cost = max([o_cost, c_cost, obs_o_cost, g_o_cost])

    if t == 1:
        return 0

    geode_scores = []

    collected = [ore + o_rob, clay + c_rob, obs + obs_rob]
    # ore += o_rob
    # clay += c_rob
    # obs += obs_rob

    #if rob["geode"]>0:
    #print(t,*collected, *robots, *costs)

    # Ore robots (enough ore to build + no more than required to build the most expensive robot)
    if ore >= o_cost and o_rob < max_ore_cost:
        build_ore = max_geodes(
            t-1, 
            collected[0] - o_cost, collected[1], collected[2],
            o_rob + 1, c_rob, obs_rob,
            *costs
        )
        geode_scores.append(build_ore)

    # Clay robots (enough ore to build + no more than required to build obsidian robot)
    if ore >= c_cost and c_rob < obs_c_cost:
        build_clay = max_geodes(
            t-1, 
            collected[0] - c_cost, collected[1], collected[2],
            o_rob, c_rob + 1, obs_rob,
            *costs
        )
        geode_scores.append(build_clay)

    # Obsidian robots (enough ore and clay to build + no more than required to build geode robot)
    if ore >= obs_o_cost and clay >= obs_c_cost and obs_rob < g_obs_cost:
        build_obsidian = max_geodes(
            t-1, 
            collected[0] -obs_o_cost, collected[1] - obs_c_cost, collected[2],
            o_rob, c_rob, obs_rob + 1,
            *costs
        )
        geode_scores.append(build_obsidian)

    # Geode robots (enough ore and obsidian to build)
    if ore >= g_o_cost and obs >= g_obs_cost:
        build_geode =  max_geodes(
            t-1, 
            collected[0] - g_o_cost, collected[1], collected[2] - g_obs_cost, 
            o_rob, c_rob, obs_rob,
            *costs
        )
        return build_geode + t-1

    # Build nothing and wait...
    build_nothing = max_geodes(t-1, *collected, *robots, *costs)
    geode_scores.append(build_nothing)

    return max(geode_scores)


# Part 1
quality = 0
for l in input:
    resources = [0,0,0]
    robots = [1,0,0]
    costs = l[1:]

    max_geodes.cache_clear()
    geodes = max_geodes(24, *resources, *robots, *costs)
    print(l[0], geodes)
    quality += l[0] * geodes

print(quality)

# Part 2
prod = 1
for l in input[:3]:
    resources = [0,0,0]
    robots = [1,0,0]
    costs = l[1:]

    max_geodes.cache_clear()
    geodes = max_geodes(32, *resources, *robots, *costs)
    print(l[0], geodes)
    prod *= geodes

print(prod)

1 2
2 0
3 3
4 3
5 5
6 4
7 1
8 4
9 2
10 0
11 6
12 5
13 1
14 3
15 2
16 9
17 4
18 7
19 0
20 2
21 0
22 1
23 0
24 8
25 2
26 16
27 7
28 0
29 1
30 0
1616
1 29
2 10
3 31
8990
